In [0]:
!pip install tensorflow==2.0.0-alpha0
!pip install tensorflow-gpu==2.0.0-alpha0

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.0.0-alpha0


In [2]:
from google.colab import drive
drive.mount('/gdrive')

import os
import sys

for root, dirs, files in os.walk('/gdrive', topdown=False):
  for name in dirs:
      if name.lower() == "lsda":  # Assumes folder named 'LSDA' to exsist, and contains the repo and data
        sys.path.append(os.path.join(root, name))

from General.Paths import Data_Path

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
from _0_DataCreation.Read_Data import load_dataframe
from General.Paths import Gitlab_Path
from Scoring.scoring_func import f1_scores_plot
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
### Import and reshape data ----------------------------------------------------------------------
fold1_df = load_dataframe(filename = 'fold1_NA_features.dat')
fold2_df = load_dataframe(filename = 'fold2_NA_features.dat')
fold3_df = load_dataframe(filename = 'fold3_NA_features.dat')

del fold1_df['id']
del fold2_df['id']
del fold3_df['id']

dw_cols = [x for x in fold1_df.columns if x[-2:] == 'dw' and x[:3] == 'pca']
fold1_df[dw_cols] = np.log(np.array(fold1_df[dw_cols]))
fold2_df[dw_cols] = np.log(np.array(fold2_df[dw_cols]))
fold3_df[dw_cols] = np.log(np.array(fold3_df[dw_cols]))
fold1_df = fold1_df.replace([-np.inf],0)
fold2_df = fold2_df.replace([-np.inf],0)
fold3_df = fold3_df.replace([-np.inf],0)


Size: 66.77Mb
Size: 80.44Mb
Size: 23.49Mb


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [0]:
### Create model ----------------------------------------------------------------------
input_length = len(fold1_df.columns) -1 #not label

inputs = tf.keras.Input(shape = (input_length,))

L1 = tf.keras.layers.Dense(128, activation = 'sigmoid', kernel_initializer=tf.initializers.VarianceScaling(scale=0.01**2))(inputs)
L12 = tf.keras.layers.Dropout(0.5)(L1)

L2 = tf.keras.layers.Dense(128,activation = 'sigmoid', kernel_initializer=tf.initializers.VarianceScaling(scale=0.01**2))(L12)
L22 = tf.keras.layers.Dropout(0.5)(L2)

L3 = tf.keras.layers.Dense(64,activation = 'sigmoid', kernel_initializer=tf.initializers.VarianceScaling(scale=0.01**2))(L22)
L32 = tf.keras.layers.Dropout(0.4)(L3)


conc_layer = tf.keras.layers.concatenate([inputs,L3])

output = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.initializers.VarianceScaling(scale=0.01**2))(conc_layer)
nn_model = tf.keras.Model(inputs=inputs, outputs=output)


In [0]:
# Define optimization algorithm
ada = tf.optimizers.Adam(lr=0.00015, decay=0.00003) #Sikkert fint

# Compile model (i.e., build compute graph)
nn_model.compile(optimizer=ada,
              loss='binary_crossentropy')


In [25]:
training_sets = [['fold1','fold2'],['fold1','fold3'],['fold2','fold3']]
val_set = ['fold3','fold2','fold1']


# Training loop
x_train1 = pd.concat([fold1_df[[x for x in fold1_df.columns if x != 'label']],fold2_df[[x for x in fold2_df.columns if x != 'label']]])
x_train2 = pd.concat([fold1_df[[x for x in fold1_df.columns if x != 'label']],fold3_df[[x for x in fold3_df.columns if x != 'label']]])
x_train3 = pd.concat([fold2_df[[x for x in fold2_df.columns if x != 'label']],fold3_df[[x for x in fold3_df.columns if x != 'label']]])

y_train1 = pd.concat([fold1_df['label'],fold2_df['label']])
y_train2 = pd.concat([fold1_df['label'],fold3_df['label']])
y_train3 = pd.concat([fold2_df['label'],fold3_df['label']])

x_trains = [x_train1,x_train2,x_train3]
y_trains = [y_train1,y_train2,y_train3]

x_val1 = fold3_df[[x for x in fold3_df.columns if x != 'label']]
x_val2 = fold2_df[[x for x in fold2_df.columns if x != 'label']]
x_val3 = fold1_df[[x for x in fold1_df.columns if x != 'label']]

y_val1 = fold3_df['label']
y_val2 = fold2_df['label']
y_val3 = fold1_df['label']

x_vals = [x_val1,x_val2,x_val3]
y_vals = [y_val1,y_val2,y_val3]

for i in range(3):
  
  x_train = x_trains[i]
  y_train = y_trains[i]
  x_val = x_vals[i]
  y_val = y_vals[i]
  callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./exciting_' + str(i) + '.hdf5', monitor='val_loss', save_best_only=True, verbose=1)
  ]

  nn_model.fit(x_train, y_train, batch_size=32, epochs=15, 
            validation_data=(x_val, y_val), validation_freq=1, 
            #steps_per_epoch=x_train.shape[0],
            callbacks=callbacks)



Train on 169254 samples, validate on 27006 samples
Epoch 1/15
169216/169254 [============================>.] - ETA: 0s - loss: 0.2568
Epoch 00001: val_loss improved from inf to 0.24412, saving model to ./exciting_0.hdf5
169254/169254 [==============================] - 19s 111us/sample - loss: 0.2568 - val_loss: 0.2441
Epoch 2/15
168896/169254 [============================>.] - ETA: 0s - loss: 0.2360
Epoch 00002: val_loss improved from 0.24412 to 0.24319, saving model to ./exciting_0.hdf5
169254/169254 [==============================] - 19s 112us/sample - loss: 0.2360 - val_loss: 0.2432
Epoch 3/15
169184/169254 [============================>.] - ETA: 0s - loss: 0.2339
Epoch 00003: val_loss improved from 0.24319 to 0.24284, saving model to ./exciting_0.hdf5
169254/169254 [==============================] - 19s 111us/sample - loss: 0.2339 - val_loss: 0.2428
Epoch 4/15
168864/169254 [============================>.] - ETA: 0s - loss: 0.2328
Epoch 00004: val_loss improved from 0.24284 to 0.24

In [27]:
test_set_df = load_dataframe(filename = 'testSet_NA_features.dat')

del test_set_df['id']
dw_cols = [x for x in test_set_df.columns if x[-2:] == 'dw' and x[:3] == 'pca']
test_set_df[dw_cols] = np.log(np.array(test_set_df[dw_cols]))
test_set_df = test_set_df.replace([-np.inf],0)

predictions = []

for i in range(3):
  print(i)
  #Get the best model
  best_model = nn_model
  best_model.load_weights('exciting_' + str(i) + '.hdf5')

  preds = best_model.predict(test_set_df).flatten()
  predictions.append(preds)

Size: 149.59Mb


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


0
1
2


In [0]:
## Make the prediction ensemble on the testSet
all_preds = np.transpose(np.array(predictions))
my_y_preds = np.mean(all_preds,axis = 1)

classifications = np.zeros(len(my_y_preds),dtype = int)
classifications[my_y_preds > 0.37] = 1

my_df = pd.DataFrame({'Id':np.arange(1,len(classifications)+1),'ClassLabel':classifications})
my_df.to_csv(Gitlab_Path + '/Ranking/eat_snow3.csv',index = False)